# Silver Datasets

## Importando as bibliotecas

In [1]:
import os
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns

from datetime import datetime

pd.set_option("display.max_columns", None)

## Definição de caminhos e diretórios

In [2]:
raw_path    = "data/raw"
bronze_path = "data/bronze"
silver_path = "data/silver"

## Helper Functions

## Criando os datasets em arquivos .csv

In [3]:
# Grava dataframe em arquivo .csv
def df_to_csv( df, path, mode ):
    with open( path, mode ) as csv_file:
        df.to_csv( csv_file, index = False, header = True )

## Calculando os indicadores

### SMA (Simple Moving Average)

In [4]:
# Simple Moving Average 
def SMA(data, ndays, _name): 
    SMA = pd.Series(data['Close'].rolling(ndays).mean(), name = _name) 
    data = data.join(SMA) 
    return data

### EWMA (Exponentially-weighted Moving Average)

In [5]:
# Calculando a EWMA
def EMA(data, col, ndays,_name): 
    EMA = pd.Series(data[col].ewm(span = ndays, min_periods = ndays - 1).mean(), name = _name) 
    data = data.join(EMA) 
    return data

### Bollinger Bands

In [6]:
# Calculando as bandas
def BBANDS(data, window):
    MA = data.Close.rolling(window).mean()
    SD = data.Close.rolling(window).std()
    data['UpperBand'] = MA + (2 * SD) 
    data['LowerBand'] = MA - (2 * SD)
    return data

### RSI (Relative Strength Index)

In [7]:
# Calculando o RSI
def rsi(close, periods = 14):
    
    close_delta = close.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

### MFI (Money Flow Index)

In [8]:
def gain(x):
    return ((x > 0) * x).sum()


def loss(x):
    return ((x < 0) * x).sum()


# Calculate money flow index
def mfi(high, low, close, volume, n=14):
    typical_price = (high + low + close)/3
    money_flow = typical_price * volume
    mf_sign = np.where(typical_price > typical_price.shift(1), 1, -1)
    signed_mf = money_flow * mf_sign
    mf_avg_gain = signed_mf.rolling(n).apply(gain, raw=True)
    mf_avg_loss = signed_mf.rolling(n).apply(loss, raw=True)
    return (100 - (100 / (1 + (mf_avg_gain / abs(mf_avg_loss))))).to_numpy()


### ATR (Average True Range)

In [9]:
def atr(high, low, close, n=14):
    tr = np.amax(np.vstack(((high - low).to_numpy(), (abs(high - close)).to_numpy(), (abs(low - close)).to_numpy())).T, axis=1)
    return pd.Series(tr).rolling(n).mean().to_numpy()

### FI (Force Index)

In [10]:
def ForceIndex(data, ndays): 
    FI = pd.Series(data['Close'].diff(ndays) * data['Volume'], name = 'ForceIndex') 
    data = data.join(FI) 
    return data

### Ease of Movement

In [11]:
# Ease of Movement
def EMV(data, ndays): 
    dm = ((data['High'] + data['Low'])/2) - ((data['High'].shift(1) + data['Low'].shift(1))/2)
    br = (data['Volume'] / 100000000) / ((data['High'] - data['Low']))
    EMV = dm / br 
    EMV_MA = pd.Series(EMV.rolling(ndays).mean(), name = 'EMV') 
    data = data.join(EMV_MA) 
    return data 

## Carregando os dados para o dataframe df_silver

In [12]:
def silver_datasets_generation():  
    file_list = []
       
    if os.path.exists( bronze_path ):
        print('entrou')
        file_list = os.listdir( bronze_path )

        for file in file_list:
            if '.csv' in file:
                print( f'Processando o arquivo {file}', os.path.getsize(f"{bronze_path}/{file}")/(1024**2), 'Mb')
                try:
                    # loadin the bronze dataset into df_silver
                    df_silver = pd.read_csv( f"{bronze_path}/{file}" )
                    
                    # sorting the df_silver by the open time
                    df_silver.sort_values( "Open_time" )
                                        
                    # converting Open_time and Close_time columns to datetime
                    df_silver[ 'Open_time' ] = df_silver[ 'Open_time' ].apply( lambda x: datetime.fromtimestamp( np.round( x/1000, 0 ) ) )
                    df_silver[ 'Close_time' ] = df_silver[ 'Close_time' ].apply( lambda x: datetime.fromtimestamp( np.round( x/1000, 0 ) ) )
                    
                    # Adding the mean price column
                    df_silver[ "Mean"] = df_silver[["Open","High","Low","Close"]].sum( axis = 1 ) / 4
                    df_silver = df_silver [["Open_time","Open","High","Low","Close","Mean","Volume","Close_time","Quote_asset_volume","Number_of_trades","Taker_buy_base_asset_volume","Taker_buy_quote_asset_volume"]]
                    
                    ########################################################################################################
                    # Including the Technical Indicators
                    ########################################################################################################

                    # Long term SMA (Simple Moving Average)
                    df_silver = SMA(df_silver, 50, 'LT_SMA')

                    # Short term SMA (Simple Moving Average)
                    df_silver = SMA(df_silver, 10, 'ST_SMA')

                    # Long term EWMA (Exponentially Moving Average)
                    df_silver = EMA(df_silver, 'Close', 26, 'LT_EMA')
                    
                    # Short term EWMA (Exponentially Moving Average)
                    df_silver = EMA(df_silver, 'Close', 12, 'ST_EMA')

                    # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
                    df_silver["MACD"] = df_silver["ST_EMA"] - df_silver["LT_EMA"]  

                    # Get the 9-Day EMA of the MACD for the Trigger line
                    df_silver = EMA(df_silver, 'MACD', 9, 'Trigger_Line')                  

                    # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
                    df_silver["MADC_H"] = df_silver["MACD"] - df_silver["Trigger_Line"]

                    # Boiler bands
                    df_silver = BBANDS(df_silver, 50)

                    # RSI (Relative Strength Index)
                    df_silver["RSI"] = rsi(df_silver["Close"])

                    # MFI (Money Flow index)
                    df_silver["MFI"] = mfi(df_silver["High"], df_silver["Low"], df_silver["Close"], df_silver["Volume"], 14)

                    # ATR (Average True Range)
                    df_silver['ATR'] = atr(df_silver['High'], df_silver['Low'], df_silver['Close'], 14)

                    # Force Index
                    df_silver = ForceIndex(df_silver, 1)

                    # EMV (Ease Movement)
                    df_silver = EMV(df_silver, 14)

                    # Dropando as linhas vazias
                    df_silver.dropna( inplace = True )

                    # reset index 
                    df_silver.reset_index( inplace = True, drop = True )
                                                          

                    # Grava dados no silver dataset
                    try:
                        if os.path.exists( silver_path ):
                            # pass
                            df_to_csv( df_silver, f"{silver_path}/{file}", 'a' )
                        
                        else:
                            os.mkdir( silver_path )
                            df_to_csv( df_silver, f"{df_silver}/{file}", 'a' )

                    except Exception as ex:
                        print( f"ERROR: {ex}" )
                        pass

                except Exception as ex:
                    print( f"ERROR: {ex}" )                
                
        return df_silver
    else:
        return None
    
df_silver = silver_datasets_generation()     
df_silver.head(100)

entrou
Processando o arquivo ETHUSDT-5m.csv 21.361140251159668 Mb
Processando o arquivo BTCUSDT-1d.csv 0.20807456970214844 Mb
Processando o arquivo BTCUSDT-1m.csv 49.28406047821045 Mb
Processando o arquivo BTCUSDT-2h.csv 1.4466238021850586 Mb
Processando o arquivo BTCUSDT-5m.csv 22.713252067565918 Mb
Processando o arquivo BUSDUSDT-1d.csv 0.12596511840820312 Mb
Processando o arquivo BUSDUSDT-1m.csv 41.569172859191895 Mb
Processando o arquivo BUSDUSDT-2h.csv 1.3057441711425781 Mb
Processando o arquivo BUSDUSDT-5m.csv 19.603483200073242 Mb
Processando o arquivo DOGEUSDT-1d.csv 0.1420278549194336 Mb
Processando o arquivo DOGEUSDT-1m.csv 47.28410530090332 Mb
Processando o arquivo DOGEUSDT-2h.csv 45.32038116455078 Mb


/tmp/ipykernel_40919/1251367882.py:13: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_silver = pd.read_csv( f"{bronze_path}/{file}" )


ERROR: '<' not supported between instances of 'int' and 'str'
Processando o arquivo DOGEUSDT-5m.csv 20.431530952453613 Mb
Processando o arquivo ETCUSDT-1d.csv 0.16420364379882812 Mb
Processando o arquivo ETCUSDT-1m.csv 38.00272560119629 Mb
Processando o arquivo ETCUSDT-2h.csv 1.2776813507080078 Mb
Processando o arquivo ETCUSDT-5m.csv 18.37930393218994 Mb
Processando o arquivo ETHUSDT-1d.csv 0.20158863067626953 Mb
Processando o arquivo ETHUSDT-1m.csv 50.84156322479248 Mb
Processando o arquivo LTCUSDT-1d.csv 0.1815013885498047 Mb
Processando o arquivo LTCUSDT-1m.csv 42.63188171386719 Mb
Processando o arquivo LTCUSDT-2h.csv 1.2812957763671875 Mb
Processando o arquivo LTCUSDT-5m.csv 18.942026138305664 Mb
Processando o arquivo MANAUSDT-1d.csv 0.08990859985351562 Mb
Processando o arquivo MANAUSDT-1m.csv 43.837979316711426 Mb
Processando o arquivo MANAUSDT-2h.csv 0.983454704284668 Mb
Processando o arquivo MANAUSDT-5m.csv 18.775035858154297 Mb
Processando o arquivo XMRUSDT-1m.csv 37.6027774810

,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,Ignore
0,2022-05-31 21:00:00,93.3,93.5,93.2,93.4,164.682,1.654042e+12,15376.8323,51.0,116.138,10844.4939,0.0
1,2022-05-31 21:01:00,93.4,93.7,93.4,93.5,195.477,1.654042e+12,18264.5980,32.0,188.862,17645.6113,0.0
2,2022-05-31 21:02:00,93.6,93.7,93.5,93.7,117.241,1.654042e+12,10970.8439,31.0,28.144,2634.5723,0.0
3,2022-05-31 21:03:00,93.7,93.8,93.7,93.8,19.300,1.654042e+12,1808.5167,12.0,19.300,1808.5167,0.0
4,2022-05-31 21:04:00,93.8,93.9,93.8,93.8,41.528,1.654042e+12,3896.6566,18.0,32.039,3006.5884,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-05-31 22:35:00,94.6,94.6,94.2,94.2,205.157,1.654047e+12,19357.7444,86.0,27.664,2606.7127,0.0
96,2022-05-31 22:36:00,94.2,94.2,94.2,94.2,21.736,1.654047e+12,2047.5312,15.0,21.736,2047.5312,0.0
97,2022-05-31 22:37:00,94.2,94.2,94.2,94.2,4.774,1.654047e+12,449.7108,8.0,4.774,449.7108,0.0
98,2022-05-31 22:38:00,94.2,94.2,94.2,94.2,25.970,1.654048e+12,2446.3740,13.0,25.970,2446.3740,0.0
